<a href="https://colab.research.google.com/github/Aditya0996/PeopleYouMightKnow-FriendRecommendation/blob/main/%E2%80%9CPeople_You_Might_Know%E2%80%9D_social_network_friendship_recommendation_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import Data and Library

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import itertools

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
from pyspark import SparkConf
conf=SparkConf().set("spark.executor.memory", "8g")

Please upload input file to sample_data folder before running!

In [ ]:
#Upload txt file under sample data
txt = spark.read.text("/content/sample_data/soc-LiveJournal1Adj.txt")

In [ ]:
txt.head(10)

[Row(value='0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(value='1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 Row(value='2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'),
 Row(value='3\t0,12,41,55,1532,12636,13185,27552,38737'),
 Row(value='4\t0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822'),
 Row(value='5\t0,1,20,2022,22939,23527,30257,32503

In [ ]:
#Add users to "user" column and friends to "friends" column
txt = txt.withColumn("user", split(txt.value, "\t")[0])
txt = txt.withColumn("friends", split(txt.value, "\t")[1])
txt.head(10)

[Row(value='0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94', user='0', friends='1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(value='1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592', user='1', friends=

In [ ]:
#Split friends column to get a list of friends
txt = txt.withColumn("friends", split(txt.friends, ","))

In [ ]:
txt = txt.select(txt.user,txt.friends)
txt.head(10)

[Row(user='0', friends=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94']),
 Row(user='1', friends=['0', '5', '20', '135', '2409', '8715', '8932', '10623', '12347', '12846', '13840', '13845', '14005', '20075', '21556', '22939', '23520', '28193', '29724', '29791', '29826', '30691', '31232', '31435', '32317', '32489', '34394', '35589', '35605', '35606', '35613', '35633', '35648', '35678', '38737', '43447', '44846', '44887', '49226', '49985', '623', '629', '4999', '6156', '13912', '14248', '15

In [ ]:
#explode the data to get user , friend
txt.select(txt.user,explode(txt.friends).alias("friend")).show(truncate=False)

+----+------+
|user|friend|
+----+------+
|0   |1     |
|0   |2     |
|0   |3     |
|0   |4     |
|0   |5     |
|0   |6     |
|0   |7     |
|0   |8     |
|0   |9     |
|0   |10    |
|0   |11    |
|0   |12    |
|0   |13    |
|0   |14    |
|0   |15    |
|0   |16    |
|0   |17    |
|0   |18    |
|0   |19    |
|0   |20    |
+----+------+
only showing top 20 rows



In [ ]:
friends = txt.select(txt.user,explode(txt.friends).alias("friend"))

In [ ]:
#Assign big value to direct friends to be able to filter direct friends out of recommendation
map_friend = friends.rdd.map(lambda row: ((row.user, row.friend),-9999999))
#Take permutation of each list of friends to make a map of mutual friends
map_mutual = txt.rdd.flatMap(lambda row: [(mutual,1) for mutual in itertools.permutations(row.friends, 2)])

Source for flatMap: https://stackoverflow.com/questions/61053329/how-to-convert-rdd-list-of-lists-into-one-list-in-pyspark

In [ ]:
map_mutual.take(10)

[(('1', '2'), 1),
 (('1', '3'), 1),
 (('1', '4'), 1),
 (('1', '5'), 1),
 (('1', '6'), 1),
 (('1', '7'), 1),
 (('1', '8'), 1),
 (('1', '9'), 1),
 (('1', '10'), 1),
 (('1', '11'), 1)]

In [ ]:
#Reduce by key to get the no. of occurance of mutual friends
map_mutual = map_mutual.reduceByKey(lambda a,b: a+b)

In [ ]:
# map_mutual.take(10)

In [ ]:
#Combining the two maps
map_mutual = map_mutual.union(map_friend)

Source for union command: https://stackoverflow.com/questions/27395420/concatenating-datasets-of-different-rdds-in-apache-spark-using-scala

In [ ]:
#Reduce by key to get "count" of direct friends go to -ve value.
reduced = map_mutual.reduceByKey(lambda a,b: a+b)

In [ ]:
#Filter out direct friends
reduced = reduced.filter(lambda x: x[1] > 0)

In [ ]:
# reduced.take(10)

In [ ]:
# ((user,mutual),count): [(user,(count,mutual))]
new_mutualFriends = reduced.map(lambda x: (x[0][0],(x[1],x[0][1])))

In [ ]:
#Group by key to get list of recommendation for each user.
new_mutualFriends = new_mutualFriends.groupByKey().mapValues(list)

In [ ]:
new_mutualFriends.take(5)

[('6121',
  [(1, '43312'),
   (2, '21253'),
   (1, '18297'),
   (2, '26055'),
   (2, '43310'),
   (1, '1687'),
   (1, '14320'),
   (1, '1244'),
   (1, '6127'),
   (2, '43155'),
   (5, '6117'),
   (4, '6109'),
   (1, '41038'),
   (1, '43300'),
   (1, '43064'),
   (1, '43360'),
   (1, '390'),
   (1, '17362'),
   (1, '13670'),
   (1, '47685'),
   (2, '30476'),
   (2, '621'),
   (1, '11062'),
   (1, '13772'),
   (1, '44181'),
   (1, '22983'),
   (1, '14349'),
   (1, '11198'),
   (1, '27148'),
   (1, '22070'),
   (1, '34688'),
   (1, '10764'),
   (1, '30741'),
   (1, '12650'),
   (1, '13233'),
   (1, '22276'),
   (1, '38802'),
   (1, '38115'),
   (2, '18357'),
   (1, '2704'),
   (1, '21776'),
   (1, '23252'),
   (1, '34288'),
   (2, '36500'),
   (2, '17052'),
   (2, '20521'),
   (1, '4875'),
   (1, '43626'),
   (1, '45839'),
   (1, '22527'),
   (1, '24243'),
   (1, '18291'),
   (1, '11045'),
   (1, '29798'),
   (1, '40337'),
   (1, '45054'),
   (2, '1433'),
   (2, '9490'),
   (1, '13621'),


Source for sorting list of tuples: https://stackoverflow.com/questions/34618029/how-to-sort-rdd-of-nested-list-structure-by-value-in-spark

In [ ]:
#Sort the count in descending order and the users with same count in ascending order.
# (k, v): (k, sorted(v, key=lambda x: x[1], reverse=True))
new_mutualFriends = new_mutualFriends.map(lambda x: (x[0], sorted(x[1], key=lambda y:(y[0], -int(y[1])), reverse=True)))

In [ ]:
new_mutualFriends.take(10)

[('21473',
  [(17, '21490'),
   (16, '20966'),
   (13, '21049'),
   (13, '21052'),
   (11, '20980'),
   (10, '21072'),
   (9, '21007'),
   (9, '21069'),
   (8, '20987'),
   (8, '21020'),
   (8, '21038'),
   (8, '21042'),
   (8, '21484'),
   (8, '21492'),
   (8, '21496'),
   (8, '23088'),
   (8, '23106'),
   (7, '21010'),
   (7, '21474'),
   (7, '21475'),
   (7, '21480'),
   (7, '21494'),
   (7, '23094'),
   (7, '23104'),
   (6, '21057'),
   (6, '21065'),
   (6, '21477'),
   (6, '21481'),
   (6, '21493'),
   (5, '20984'),
   (5, '21027'),
   (5, '21028'),
   (5, '21488'),
   (5, '21489'),
   (4, '20988'),
   (4, '20994'),
   (4, '20999'),
   (4, '21037'),
   (4, '21054'),
   (4, '21056'),
   (4, '21061'),
   (4, '21495'),
   (4, '21497'),
   (4, '21498'),
   (4, '23091'),
   (4, '23097'),
   (3, '21012'),
   (3, '21059'),
   (3, '21469'),
   (3, '21472'),
   (3, '23103'),
   (2, '10954'),
   (2, '20969'),
   (2, '20970'),
   (2, '20972'),
   (2, '20977'),
   (2, '20979'),
   (2, '20981'

In [ ]:
#Take the top 20 recommendation for each user
n = 20
new_mutualFriends = new_mutualFriends.map(lambda x:(x[0],x[1][:n]))

In [ ]:
recommendations = new_mutualFriends.map(lambda x: (x[0],[y[1] for y in x[1]]))

In [ ]:
# Get the recommendations as a list
recommendationList = recommendations.collect()

In [ ]:
#Find users with no friends or no mutual friends by comparing it with original file
txt1 = spark.read.text("/content/sample_data/soc-LiveJournal1Adj.txt")
txt1 = txt1.withColumn("values", split(txt1.value, "\t")[0])
empty = txt1.rdd.map(lambda row: (row.values,1))
notEmpty = recommendations.map(lambda row: (row[0],1))
fullList = empty.union(notEmpty)
emptyList = fullList.reduceByKey(lambda a,b: a-b).filter(lambda x: x[1]>0).map(lambda x: (x[0],[])).collect()

In [ ]:
# Add the empty list and recommendation list to get the final list. Also, sorting the list with users in ascending order.
def getkey(x):
  return int(x[0])
finalRecommendation = recommendationList + emptyList
finalRecommendation.sort(key= getkey)

In [ ]:
len(finalRecommendation)

49995

In [ ]:
for x in finalRecommendation:
  if int(x[0]) == 11 or int(x[0]) == 924 or int(x[0]) == 8941 or int(x[0]) == 8942 or int(x[0]) == 9019 or int(x[0]) == 9020 or int(x[0]) == 9021 or int(x[0]) == 9022 or int(x[0]) == 9990 or int(x[0]) == 9992 or int(x[0]) == 9993:
    print (x[0],"\t",','.join(x[1]))

11 	 27552,7785,27573,27574,27589,27590,27600,27617,27620,27667,32072,33192,10,12,110,638,1797,2141,5784,6893
924 	 439,2409,6995,11860,15416,43748,45881
8941 	 8943,8944,8940
8942 	 8939,8940,8943,8944
9019 	 9022,317,9023
9020 	 9021,9016,9017,9022,317,9023
9021 	 9020,9016,9017,9022,317,9023
9022 	 9019,9020,9021,317,9016,9017,9023
9990 	 13134,13478,13877,34299,34485,34642,37941
9992 	 9987,9989,35667,9991
9993 	 9991,13134,13478,13877,34299,34485,34642,37941


In [ ]:
#Get the output.txt file with all recommendations
f = open('/content/sample_data/output.txt', 'w')
for line in finalRecommendation:
  f.write(""+line[0]+"\t"+','.join(line[1])+"\n")